In [ ]:
# cell-collective-control-kernels.ipynb
#
# Bryan Daniels
# 2023/11/2
#
# Computing control kernels for relevant networks in the cell collective database.
#

In [44]:
from neet.boolean import LogicNetwork
from neet.controlkernel.control_kernel_analysis import ck_analysis
from neet.controlkernel.modularity import pinning_produces_desired_attractor
import numpy as np
import glob
import os
from tqdm import tqdm

# Load networks from Cell Collective

In [6]:
def load_cell_collective_network(directory):
    """
    Creates neet network from cell collective data in the given directory.
    """
    expressions_file = '{}/expressions.txt'.format(directory)
    external_file = '{}/external.txt'.format(directory)
    net = LogicNetwork.read_logic(expressions_file, external_file)
    return net

In [4]:
cell_collective_directory = '../Data/Cell Collective/'

In [18]:
# load all cell collective networks into a dictionary
net_dict = {}
skip_network = 'ErbB_Receptor_Signaling' # skip this network because it is too large
for network_directory in tqdm(glob.glob(cell_collective_directory+"/*")):
    if os.path.isdir(network_directory) and not network_directory.endswith(skip_network):
        #print("Loading network from directory {}".format(network_directory))
        net_name = os.path.split(network_directory)[-1]
        net = load_cell_collective_network(network_directory)
        #print("net_name = {}".format(net_name))
        net_dict[net_name] = net

100%|███████████████████████████████████████████| 72/72 [00:09<00:00,  7.91it/s]


# Do example control kernel analysis

In [21]:
# control kernel analysis on an example network
net = net_dict['Cortical_Area_Development']

In [47]:
# the default code finds a single control kernel (of minimal size) for each attractor
ck_data = ck_analysis(net)
attractor_list = ck_data['attractors']
ck_list = ck_data['control_kernels']
for att,ck in zip(attractor_list,ck_list):
    ck_names = [ net.names[i] for i in ck ]
    print('Control kernel for attractor {}:'.format(np.transpose(net.decode(att))))
    print('    {}'.format(ck_names))

Finding attractors and control kernels...
Control kernel for attractor [[1 1 0 0 0]]:
    ['Emx2']
Control kernel for attractor [[0 0 1 1 1]]:
    ['Fgf8']


In [34]:
# code for finding all possible control kernels of size 1 
# (copied from Control-of-Synthetic-Boolean-Networks/code/BoolODE_data.ipynb)
def find_all_control_kernels_of_size_one(net):
    attractors = net.attractors
    for att_index in range(len(attractors)):
        desired_attractor = attractors[att_index].copy()
        desired_attractor_decoded = [net.decode(state) for state in desired_attractor]
        print("Control kernels of size one for attractor {}".format(desired_attractor_decoded))
        for i,name in enumerate(net.names):
            pin = [i,]
            pin_states = [[desired_attractor_decoded[0][i],],]
            if pinning_produces_desired_attractor(net,pin,pin_states,desired_attractor):
                print({name})

In [36]:
find_all_control_kernels_of_size_one(net)

Control kernels of size one for attractor [[1, 1, 0, 0, 0]]
{'Emx2'}
{'Fgf8'}
{'Sp8'}
Control kernels of size one for attractor [[0, 0, 1, 1, 1]]
{'Fgf8'}


# Find Cell Collective networks that have only two attractors 

In [ ]:
two_attractor_names = []
for network_name in net_dict.keys():
    print("Finding attractors for {}...".format(network_name))
    net = net_dict[network_name]
    try:
        print("    Number of attractors = {}".format(len(net.attractors)))
        if len(net.attractors) == 2:
            two_attractor_names.append(network_name)
    except ValueError:
        print("    Error")    

Finding attractors for Arabidopsis_Thaliana_Cell_Cycle...
    Number of attractors = 1
Finding attractors for T-LGL_Survival_Network_2008...
    Error
Finding attractors for Wg_Pathway_Of_Drosophila_Signalling_Pathways...
